In [2]:
import numpy as np
import pandas as pd

In [25]:
data = pd.read_csv("c:/Users/tquan/Downloads/ner_dataset.csv", encoding= 'latin1')
data = data.fillna(method='ffill')
data

C:\Users\tquan\AppData\Local\Temp\ipykernel_10856\213452595.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method='ffill')


,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O
...,...,...,...,...
1048570,Sentence: 47959,they,PRP,O
1048571,Sentence: 47959,responded,VBD,O
1048572,Sentence: 47959,to,TO,O
1048573,Sentence: 47959,the,DT,O


In [1]:
import pandas as pd
from typing import List

class Vocab:
    def __init__(self, data_path):

        self.word_to_idx = {}
        self.idx_to_word = {}

        self.tag_to_idx = {}
        self.idx_to_tag = {}

        self.dataset_path = data_path

        self.build_vocab()

    def build_words_and_tags(self):
        dataset = pd.read_csv(self.dataset_path, encoding= 'latin1')
        dataset = dataset.fillna(method= 'ffill')
        
        list_words = list(set(dataset["Word"].values))
        list_words.append('<pad>')

        # lis tags
        list_tags = list(set(dataset["Tag"].values))

        return list_words, list_tags
    
    def build_vocab(self):
        list_words, list_tags = self.build_words_and_tags()
        
        self.word_to_idx = {word: idx + 1 for idx, word in enumerate(list_words)}
        self.idx_to_word = {idx: word for word, idx in self.word_to_idx.items()}

        self.tag_to_idx = {tag: idx + 1 for idx, tag in enumerate(list_tags)}
        self.idx_to_tag = {idx: tag for tag, idx in self.tag_to_idx.items()}

    def convert_tokens_to_ids(self, tokens: List):
        return [self.word_to_idx.get(token) for token in tokens]
    
    def convert_ids_to_tokens(self, ids: List):
        return [self.idx_to_word[idx] for idx in ids]
    
    def convert_tags_to_ids(self, tags: List):
        return [self.tag_to_idx.get(tag) for tag in tags]
    
    def convert_ids_to_tags(self, ids: List):
        return [self.idx_to_tag[idx] for idx in ids]
    
    def vocab_size(self):
        return len(self.word_to_idx) + 1
    
    def num_tags(self):
        return len(self.tag_to_idx)
    
    def pad_token_idx(self):
        return 0

In [2]:
from torch.utils.data import Dataset, DataLoader

import pandas as pd
from torch.utils.data import random_split

class MyDataset(Dataset):
    def __init__(self, dataset_path: str, vocab: Vocab):
        super(MyDataset, self).__init__()

        self.sentence = []
        self.tag = []
        self.vocab = vocab

        data = pd.read_csv(dataset_path, encoding= 'latin1')
        data = data.fillna(method= 'ffill')

        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]

        self.grouped = data.groupby("Sentence #").apply(agg_func)
        self.obj = [s for s in self.grouped]

        self.sentences = [[w[0] for w in s] for s in self.obj]
        self.tags = [[w[2] for w in s] for s in self.obj]

        self.sentence = [self.vocab.convert_tokens_to_ids(s) for s in self.sentences]
        self.tag = [self.vocab.convert_tags_to_ids(t) for t in self.tags]

    def __len__(self):
        return len(self.sentence)
    
    def __getitem__(self, index):
        return {
            "sentence": self.sentence[index],
            "tag": self.tag[index]
        }

In [3]:
vocab = Vocab("c:/Users/tquan/Downloads/ner_dataset.csv")

C:\Users\tquan\AppData\Local\Temp\ipykernel_9476\2718398789.py:19: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dataset = dataset.fillna(method= 'ffill')


In [4]:
data = MyDataset("c:/Users/tquan/Downloads/ner_dataset.csv", vocab= vocab)
data

C:\Users\tquan\AppData\Local\Temp\ipykernel_9476\131286098.py:15: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method= 'ffill')


In [7]:
data.sentences

[['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-geo',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-geo',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-gpe',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['B-gpe',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-tim',
  'O',
  'O',
  'O',
  'B-org',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'B-tim',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-geo',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-org',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-geo',
  'I-geo',
  'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['B-geo',
  'O',
  'O',
  'B-per',
  'I-per',
  'O',
  'B-tim',
  'O',
  'B-geo',
  'O',
  'B-gpe',
  'O',
  'B-gpe',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-geo',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['B-per',
  'I-per',
  'O',
  'O',
  'O',
  'O',
